In [20]:
import mido
import numpy as np
from mido import Message, MidiFile, MidiTrack
import tensorflow as tf
import numpy as np
import json
import os
import pretty_midi
import pickle

### New Index Reference 
- training_data[n] selects song at index n 
- Training_data[n][indicies below]

- index 0: 'loudness / segments' loudness level
- index 1: 'beats' start binary
- index 2: 'beats' duration binary
- index 3: 'tatums' start binary
- index 4: 'tatums' duration binary
- index 5: 'bars' start binary
- index 6: 'bars' duration binary
- index 7: 'sections' start binary

In [2]:
def pull_json_files(epoch_number):
    f = open('../Data/Song_Outputs/generated_songs_{}.json'.format(epoch_number))
    data = json.load(f)  
    return np.array(data).squeeze()

In [35]:
with open('padded_training_data.pickle', 'rb') as f:
    training_data = pickle.load(f)
training_data = np.array(training_data[1])

In [29]:
def create_TS_map(duration_ts, sample_rate=10):
    #sample_rate is in samples per second, which get converted to ms in the method
    sample_rate = sample_rate / 1000
    # ms
    step = 1/sample_rate
    time_row = np.arange(start=step, stop = duration_ts, step = step)
    return time_row

In [30]:
def velocity_helper(input):
    return int(max(min(input,127),0))

In [37]:
def create_midi_file(time_series_labels, beats, tatums, loudness, epoch, song_number):
    # Create a PrettyMIDI object
    mid = pretty_midi.PrettyMIDI()

    # Create a Piano instrument
    piano = pretty_midi.Instrument(program=pretty_midi.instrument_name_to_program('Acoustic Grand Piano'))
    loudness = np.array(loudness)


    # Add the notes based on the time_series_labels
    for idx in range(len(time_series_labels)):
        time = time_series_labels[idx]
        beat = beats[idx]
        tatum = tatums[idx]
        loud = loudness[idx]
        
        # Normalize the loudness values to the range [0, 1]
        # will be used for 'velocity' as it's named in the music space

        if (np.max(loudness) - np.min(loudness)) == 0:
            loud = 100
        else:
            loud = velocity_helper(loud - np.min(loudness)) / (np.max(loudness) - np.min(loudness))
        
        # If the beat is present, add a note with the loudness as velocity
        if beat == 1.0:

            velocity = loud
            assert 0 < velocity < 127
            pitch = 60
            start= time / 1000
            end=(time + 50) / 1000

            
            note = pretty_midi.Note(velocity=velocity, pitch=pitch, start=start, end=end)
            piano.notes.append(note)
        
        # If the tatum is present, add a note with the tatum number as the pitch
        if tatum == 1.0:
            tatum_pitch = int(64 + (idx % 4))
            note = pretty_midi.Note(velocity=loud, pitch=tatum_pitch, start=time / 1000, end=(time + 50) / 1000)
            piano.notes.append(note)
    
    # Add the instrument to the midi object
    mid.instruments.append(piano)
    
    # Set the tempo
    tempo = mid.estimate_tempo()
    mid.tempo = tempo
    
    # Save the midi file
    mid.write('../Data/Midi_files/Epoch_{}_Midi_{}.mid'.format(epoch,song_number))


In [38]:
def create_three_midis(epoch, clip_point,is_generated=True):
    time_series_data = create_TS_map(3000, sample_rate=10)
    if is_generated:
        three_songs = pull_json_files(epoch)
    else:
        three_songs = []
        for i in range(2):
            three_songs.append(np.array(training_data[np.random.randint(0,6000)]).squeeze())

    count = 1
    for song in three_songs:
        beats = song[1]
        tatums = song[3]
        loudness = song[0]
        create_midi_file(time_series_labels=time_series_data[:clip_point],beats=beats[:clip_point],tatums=tatums[:clip_point],loudness=loudness[:clip_point],epoch=epoch,song_number=count)
        count += 1

In [41]:
create_three_midis(99,3000,True)